1. Из ноутбуков по практике "Рекуррентные и одномерные сверточные нейронные сети" выберите лучшую сеть, либо создайте свою.
2. Запустите раздел "Подготовка"
3. Подготовьте датасет с параметрами `VOCAB_SIZE=20'000`, `WIN_SIZE=1000`, `WIN_HOP=100`, как в ноутбуке занятия, и обучите выбранную сеть. Параметры обучения можно взять из практического занятия. Для  всех обучаемых сетей в данной работе они должны быть одни и теже.
4. Поменяйте размер словаря tokenaizera (`VOCAB_SIZE`) на `5000`, `10000`, `40000`.  Пересоздайте датасеты, при этом оставьте `WIN_SIZE=1000`, `WIN_HOP=100`.
Обучите выбранную нейронку на этих датасетах.  Сделайте выводы об  изменении  точности распознавания авторов текстов. Результаты сведите в таблицу
5. Поменяйте длину отрезка текста и шаг окна разбиения текста на векторы  (`WIN_SIZE`, `WIN_HOP`) используя значения (`500`,`50`) и (`2000`,`200`). Пересоздайте датасеты, при этом оставьте `VOCAB_SIZE=20000`. Обучите выбранную нейронку на этих датасетах. Сделайте выводы об  изменении точности распознавания авторов текстов.

# Импорт библиотек и настройка окружения
В этом блоке производится импорт всех необходимых библиотек, которые будут использоваться в процессе выполнения задания.

Работа с данными и массивами:

- numpy — операции с массивами и числовыми данными.

- utils — для работы с one-hot кодированием.

Нейросетевые модели (Keras):

- Sequential, Dense, Dropout, Embedding и другие — для построения различных типов моделей: полносвязных, рекуррентных (RNN, LSTM, GRU), сверточных (Conv1D).

- Bidirectional — для двунаправленных рекуррентных слоев.

Текстовая предобработка:

- Tokenizer — для преобразования текстов в последовательности индексов.

Визуализация:

- matplotlib.pyplot — построение графиков.

- ConfusionMatrixDisplay — отображение матрицы ошибок.

- %matplotlib inline — отображение графиков прямо в ноутбуке.

- plot_model — визуализация архитектуры сети.

Прочие утилиты:

- gdown — загрузка данных из Google-диска.

- os, re, time — работа с файлами, временем и регулярными выражениями.

- display — удобный вывод объектов в Jupyter/Colab.



In [ ]:
# Стандартная библиотека
import os
import re
import time
import warnings
import zipfile  # Для работы с архивами

# Сторонние библиотеки
import gdown
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from tensorflow.keras import utils
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Bidirectional,
    Conv1D,
    Dense,
    Dropout,
    Embedding,
    Flatten,
    GlobalMaxPooling1D,
    GRU,
    LSTM,
    MaxPooling1D,
    SimpleRNN,
    SpatialDropout1D,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model

# Подавление предупреждений
warnings.filterwarnings("ignore")

# Отображение графиков в ноутбуке
%matplotlib inline

#  Загрузка и распаковка датасета
В этом блоке осуществляется автоматическая загрузка архива writers.zip из облачного хранилища Яндекс и его последующая распаковка в папку writers/. Полученные данные будут использоваться для дальнейшей подготовки выборки и обучения модели на текстах русских писателей.

In [ ]:
if not os.path.exists('writers.zip'):
    # Загрузим датасет из облака
    gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip', None, quiet=True)


'writers.zip'

In [ ]:
# Распакуем архив в папку writers
if not os.path.exists('writers'):
    with zipfile.ZipFile('writers.zip', 'r') as zip_ref:
        zip_ref.extractall('writers')

# Выводим список файлов в папке writers
print("\n".join(os.listdir('writers')))

Archive:  writers.zip
  inflating: writers/(Клиффорд_Саймак) Обучающая_5 вместе.txt  
  inflating: writers/(Клиффорд_Саймак) Тестовая_2 вместе.txt  
  inflating: writers/(Макс Фрай) Обучающая_5 вместе.txt  
  inflating: writers/(Макс Фрай) Тестовая_2 вместе.txt  
  inflating: writers/(О. Генри) Обучающая_50 вместе.txt  
  inflating: writers/(О. Генри) Тестовая_20 вместе.txt  
  inflating: writers/(Рэй Брэдберри) Обучающая_22 вместе.txt  
  inflating: writers/(Рэй Брэдберри) Тестовая_8 вместе.txt  
  inflating: writers/(Стругацкие) Обучающая_5 вместе.txt  
  inflating: writers/(Стругацкие) Тестовая_2 вместе.txt  
  inflating: writers/(Булгаков) Обучающая_5 вместе.txt  
  inflating: writers/(Булгаков) Тестовая_2 вместе.txt  


**Настройка констант для структуры данных**

В этом блоке задаются основные константы, необходимые для корректной загрузки и обработки данных:

- FILE_DIR — имя директории, в которой находятся текстовые файлы с данными.

- SIG_TRAIN и SIG_TEST — сигнатуры (подстроки) в названиях файлов, по которым будет происходить разделение на обучающую и тестовую выборки.


In [ ]:
# Настройка констант для загрузки данных
FILE_DIR  = 'writers'                     # Папка с текстовыми файлами
SIG_TRAIN = 'обучающая'                   # Признак обучающей выборки в имени файла
SIG_TEST  = 'тестовая'                    # Признак тестовой выборки в имени файла

# Загрузка и распределение текстов по классам и выборкам
В этом блоке происходит загрузка текстов из файлов и их распределение по классам и типам выборок:

- Сначала извлекаются все имена файлов из указанной директории FILE_DIR.

- С помощью регулярного выражения из имени файла извлекается имя класса и тип выборки (обучающая или тестовая).

- Для каждого уникального класса создаются отдельные ячейки в списках text_train и text_test.

- Затем тексты из файлов добавляются к соответствующему классу и выборке (обучающей или тестовой).

В результате формируются структурированные списки текстов по классам для дальнейшей обработки.

In [ ]:
# Подготовим пустые списки
CLASS_LIST = []  # Список классов
text_train = []  # Список для оучающей выборки
text_test = []   # Список для тестовой выборки

# Получим списка файлов в папке
file_list = os.listdir(FILE_DIR)

for file_name in file_list:
    # Выделяем имя класса и типа выборки из имени файла
    m = re.match('\((.+)\) (\S+)_', file_name)
    # Если выделение получилось, то файл обрабатываем
    if m:

        # Получим имя класса
        class_name = m[1]

        # Получим имя выборки
        subset_name = m[2].lower()

        # Проверим тип выборки
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name

        # Если тип выборки обучающая либо тестовая - файл обрабатываем
        if is_train or is_test:

            # Добавляем новый класс, если его еще нет в списке
            if class_name not in CLASS_LIST:
                print(f'Добавление класса "{class_name}"')
                CLASS_LIST.append(class_name)

                # Инициализируем соответствующих классу строки текста
                text_train.append('')
                text_test.append('')

            # Найдем индекс класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)
            print(f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.')

            # Откроем файл на чтение
            with open(f'{FILE_DIR}/{file_name}', 'r') as f:

                # Загрузим содержимого файла в строку
                text = f.read()
            # Определим выборку, куда будет добавлено содержимое
            subset = text_train if is_train else text_test

            # Добавим текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += ' ' + text.replace('\n', ' ')

Добавление класса "Макс Фрай"
Добавление файла "(Макс Фрай) Обучающая_5 вместе.txt" в класс "Макс Фрай", обучающая выборка.
Добавление класса "Рэй Брэдберри"
Добавление файла "(Рэй Брэдберри) Тестовая_8 вместе.txt" в класс "Рэй Брэдберри", тестовая выборка.
Добавление класса "Клиффорд_Саймак"
Добавление файла "(Клиффорд_Саймак) Тестовая_2 вместе.txt" в класс "Клиффорд_Саймак", тестовая выборка.
Добавление класса "О. Генри"
Добавление файла "(О. Генри) Тестовая_20 вместе.txt" в класс "О. Генри", тестовая выборка.
Добавление файла "(О. Генри) Обучающая_50 вместе.txt" в класс "О. Генри", обучающая выборка.
Добавление класса "Стругацкие"
Добавление файла "(Стругацкие) Тестовая_2 вместе.txt" в класс "Стругацкие", тестовая выборка.
Добавление класса "Булгаков"
Добавление файла "(Булгаков) Обучающая_5 вместе.txt" в класс "Булгаков", обучающая выборка.
Добавление файла "(Булгаков) Тестовая_2 вместе.txt" в класс "Булгаков", тестовая выборка.
Добавление файла "(Клиффорд_Саймак) Обучающая_5 вмест

**Определение количества классов**

После загрузки и распределения текстов определяется общее количество уникальных классов (авторов). Это необходимо для последующих шагов, включая one-hot кодирование меток и настройку выходного слоя нейросети.

- CLASS_COUNT — число уникальных классов, равное длине списка CLASS_LIST.

In [ ]:
# Определим количество классов
CLASS_COUNT = len(CLASS_LIST)

**Вывод списка классов**

Этот блок выводит перечень всех уникальных классов (авторов), чьи тексты были загружены из датасета.

In [ ]:
# Выведем прочитанные классы текстов
print(CLASS_LIST)

['Макс Фрай', 'Рэй Брэдберри', 'Клиффорд_Саймак', 'О. Генри', 'Стругацкие', 'Булгаков']


**Подсчет количества обучающих текстов**

Блок выводит количество обучающих текстов, хранящихся в списке text_train.
Каждый элемент этого списка соответствует одному классу (автору), и содержит объединённый текст всех его обучающих файлов.

In [ ]:
# Посчитаем количество текстов в обучающей выборке
print(len(text_train))

6


**Просмотр фрагментов текстов по классам**

Цикл перебирает все классы (авторов) в датасете и выводит:

- Название каждого класса.

- Первые 200 символов из обучающего текста (text_train) этого класса.

- Первые 200 символов из тестового текста (text_test) этого класса.

Это позволяет убедиться, что данные загружены корректно и дают общее представление о содержимом выборок.


In [ ]:
# Проверим загрузки: выведем начальные отрывки из каждого класса

for cls in range(CLASS_COUNT):                   # Запустим цикл по числу классов
    print(f'Класс: {CLASS_LIST[cls]}')           # Выведем имя класса
    print(f'  train: {text_train[cls][:200]}')   # Выведем фрагмент обучающей выборки
    print(f'  test : {text_test[cls][:200]}')    # Выведем фрагмент тестовой выборки
    print()

Класс: Макс Фрай
  train:  ﻿Власть несбывшегося   – С тех пор как меня угораздило побывать в этой грешной Черхавле, мне ежедневно снится какая-то дичь! – сердито сказал я Джуффину. – Сглазили они меня, что ли? А собственно, по
  test :  ﻿Слишком много кошмаров    Когда балансируешь над пропастью на узкой, скользкой от крови доске, ответ на закономерный вопрос: «Как меня сюда занесло?» – вряд ли принесёт практическую пользу. Зато пои

Класс: Рэй Брэдберри
  train:  ﻿451° по Фаренгейту   ДОНУ КОНГДОНУ С БЛАГОДАРНОСТЬЮ   Если тебе дадут линованную бумагу, пиши поперёк.  Хуан Рамон Хименес   Часть 1  ОЧАГ И САЛАМАНДРА   Жечь было наслаждением. Какое-то особое насл
  test :  ﻿Марсианские хроники   МОЕЙ ЖЕНЕ МАРГАРЕТ С ИСКРЕННЕЙ ЛЮБОВЬЮ   «Великое дело – способность удивляться, – сказал философ. – Космические полеты снова сделали всех нас детьми».   Январь 1999  Ракетное 

Класс: Клиффорд_Саймак
  train:  ﻿Всё живое...     Когда я выехал из нашего городишка и повернул на шоссе, позади оказал

**Контекстный менеджер для измерения времени выполнения**

Создается класс timex, реализующий контекстный менеджер для удобного измерения времени выполнения блока кода.

- При входе (__enter__) фиксируется текущее время.

- При выходе (__exit__) выводится время, затраченное на выполнение блока.


In [ ]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with

class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        print('Время обработки: {:.2f} с'.format(time.time() - self.t))

# Настройка параметров эксперимента
Задаются параметры для экспериментов с различными размерами словаря токенизатора и параметрами окна сегментации текста:

- VOCAB_SIZES — список значений размера словаря для токенизатора (от 5000 до 40000).

- WIN_SIZES — список длин окон (фрагментов текста) для разбиения.

- WIN_HOPS — соответствующие шаги (смещения) окна при сегментации текста.

Также фиксируются параметры обучения нейронных сетей:

- число эпох (EPOCHS) — 5,

- размер батча (BATCH_SIZE) — 128.

In [ ]:
VOCAB_SIZES = [5000, 10000, 20000, 40000]  # изменение размера словаря токенизатора
WIN_SIZES = [1000, 500, 2000]   # Размеры окна сегментации текста
WIN_HOPS  = [100, 50, 200]  # Шаг окна сегментации текста

EPOCHS = 5
BATCH_SIZE = 128

# Подготовка датасета с параметрами
Функция prepare_dataset отвечает за подготовку обучающих и тестовых данных для нейросети с учетом заданных параметров: размера словаря (vocab_size), длины окна сегментации текста (win_size) и шага окна (win_hop).

Внутри функции:

- создаётся токенизатор с ограничением словаря по размеру и обработкой неизвестных слов;

- выполняется разбиение каждого текста из обучающей и тестовой выборки на последовательности фиксированной длины с заданным шагом;

- применяется паддинг последовательностей до фиксированной длины окна;

- метки классов преобразуются в формат one-hot encoding;

- возвращаются подготовленные массивы признаков и меток для обучения и тестирования, а также сам токенизатор.

Этот блок позволяет гибко формировать датасеты под разные параметры эксперимента с размерами словаря и окон сегментации.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Функция подготовки датасета
# Подготовка датасета с разными VOCAB_SIZE, WIN_SIZE, WIN_HOP.
def prepare_dataset(vocab_size, win_size, win_hop):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
    tokenizer.fit_on_texts(text_train + text_test)

    # Разбивка текста на отрезки фиксированной длины с заданным шагом.
    # Реализация параметров WIN_SIZE, WIN_HOP.
    def split_texts(texts):
        sequences = []
        labels = []
        for idx, text in enumerate(texts):
            seq = tokenizer.texts_to_sequences([text])[0]
            for i in range(0, len(seq) - win_size, win_hop):
                chunk = seq[i:i+win_size]
                sequences.append(chunk)
                labels.append(idx)
        return sequences, labels

    # Подготовка входных и выходных данных
    # Паддинг последовательностей и one-hot-кодировка меток.
    X_train, y_train = split_texts(text_train)
    X_test, y_test = split_texts(text_test)

    X_train = pad_sequences(X_train, maxlen=win_size)
    X_test = pad_sequences(X_test, maxlen=win_size)

    y_train = utils.to_categorical(y_train, CLASS_COUNT)
    y_test = utils.to_categorical(y_test, CLASS_COUNT)

    return np.array(X_train), np.array(X_test), y_train, y_test, tokenizer

#  Функция построения рекуррентной нейронной сети
Функция build_model создаёт модель последовательной нейронной сети с использованием слоев:

- Embedding — преобразует входные индексы слов в векторное представление размерности 128, с длиной входной последовательности win_size.

- SpatialDropout1D — применяется дропаут для регуляризации, предотвращая переобучение.

- Bidirectional LSTM — двухнаправленная LSTM с 64 нейронами, которая учитывает контекст текста в обеих направлениях; return_sequences=False означает, что выводится только последний временной шаг.

- Dropout — дополнительный слой дропаут для регуляризации.

- Dense с функцией активации softmax — классификатор на количество классов CLASS_COUNT.

Модель компилируется с функцией потерь categorical_crossentropy, оптимизатором adam и метрикой точности accuracy.

In [ ]:
def build_model(vocab_size, win_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length=win_size))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dropout(0.5))
    model.add(Dense(CLASS_COUNT, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Изменение размера словаря (VOCAB_SIZE) и обучение модели
В этом блоке происходит экспериментальное обучение модели при разных размерах словаря токенизатора (VOCAB_SIZE):

- Для каждого значения из списка VOCAB_SIZES (5000, 10000, 20000, 40000):

- Подготавливается датасет с фиксированной длиной окна WIN_SIZE=1000 и шагом окна WIN_HOP=100 с помощью функции prepare_dataset.

- Строится и компилируется модель build_model с текущим размером словаря.

- Модель обучается в течение EPOCHS эпох, с размером батча BATCH_SIZE. Обучение проводится без подробного вывода (verbose=0).

- По окончании обучения оценивается точность модели на тестовом наборе. Результаты (VOCAB_SIZE и точность) сохраняются.

- Время каждого цикла измеряется с помощью контекстного менеджера timex.

В конце выводится итоговая таблица с точностью классификации для каждого размера словаря.



In [ ]:
results_vocab = []

# Задание: VOCAB_SIZE изменение
for vocab_size in VOCAB_SIZES:
    print(f'\n\n=== VOCAB_SIZE = {vocab_size} ===')
    with timex():
        X_train, X_test, y_train, y_test, _ = prepare_dataset(vocab_size, 1000, 100)
        model = build_model(vocab_size, 1000)
        history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
                            validation_data=(X_test, y_test), verbose=0)
        acc = model.evaluate(X_test, y_test, verbose=0)[1]
        results_vocab.append((vocab_size, acc))
        print(f'Accuracy: {acc:.4f}')

# Таблица результатов по VOCAB_SIZE
print("\nТочность для разных размеров словаря:")
for vocab_size, acc in results_vocab:
    print(f'VOCAB_SIZE={vocab_size:<6} --> Accuracy={acc:.4f}')



=== VOCAB_SIZE = 5000 ===
Accuracy: 0.6775
Время обработки: 118.22 с


=== VOCAB_SIZE = 10000 ===
Accuracy: 0.6081
Время обработки: 118.03 с


=== VOCAB_SIZE = 20000 ===
Accuracy: 0.6326
Время обработки: 117.60 с


=== VOCAB_SIZE = 40000 ===
Accuracy: 0.7150
Время обработки: 118.81 с

Точность для разных размеров словаря:
VOCAB_SIZE=5000   --> Accuracy=0.6775
VOCAB_SIZE=10000  --> Accuracy=0.6081
VOCAB_SIZE=20000  --> Accuracy=0.6326
VOCAB_SIZE=40000  --> Accuracy=0.7150


**Изменение параметров окна сегментации текста (WIN_SIZE, WIN_HOP)**

В этом блоке исследуется влияние длины окна (WIN_SIZE) и шага окна (WIN_HOP) на качество классификации:

- Для каждой пары параметров (WIN_SIZE, WIN_HOP) из списка [(500, 50), (2000, 200)]:

- Готовится датасет с фиксированным размером словаря VOCAB_SIZE=20000, а длина и шаг окна задаются текущими значениями.

- Строится и компилируется модель с текущей длиной окна.

- Модель обучается в течение заданного количества эпох (EPOCHS) и размера батча (BATCH_SIZE).

- После обучения вычисляется точность модели на тестовом наборе, и результаты сохраняются.

- Время подготовки и обучения фиксируется с помощью контекстного менеджера timex.

Итогом работы блока является таблица с точностью классификации для каждого варианта параметров окна.

In [ ]:
results_win = []

for win_size, win_hop in [(500,50), (2000,200)]:
    print(f'\n\n=== WIN_SIZE = {win_size}, WIN_HOP = {win_hop} ===')
    with timex():
        X_train, X_test, y_train, y_test, _ = prepare_dataset(20000, win_size, win_hop)
        model = build_model(20000, win_size)
        history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
                            validation_data=(X_test, y_test), verbose=0)
        acc = model.evaluate(X_test, y_test, verbose=0)[1]
        results_win.append((win_size, win_hop, acc))
        print(f'Accuracy: {acc:.4f}')

# Таблица результатов по WIN_SIZE/WIN_HOP
print("\nТочность для разных параметров окна:")
for win_size, win_hop, acc in results_win:
    print(f'WIN_SIZE={win_size:<5}, WIN_HOP={win_hop:<4} --> Accuracy={acc:.4f}')



=== WIN_SIZE = 500, WIN_HOP = 50 ===
Accuracy: 0.7291
Время обработки: 119.90 с


=== WIN_SIZE = 2000, WIN_HOP = 200 ===
Accuracy: 0.4405
Время обработки: 118.99 с

Точность для разных параметров окна:
WIN_SIZE=500  , WIN_HOP=50   --> Accuracy=0.7291
WIN_SIZE=2000 , WIN_HOP=200  --> Accuracy=0.4405


# Вывод по заданию
Отсюда по заданию видно, что лучший результат даёт небольшое окно (WIN_SIZE=500) с малым шагом (WIN_HOP=50), то есть это даёт больше обучающих примеров, а также модель чаще видит начало и конец текста, и повышается устойчивость и обобщаемость.

Длинные окна (WIN_SIZE=2000) ухудшают результат, так как такие окна покрывают слишком большую часть текста, их меньше по количеству, а также труднее выделить локальные стилистические признаки автора. Так же возможна перегрузка модели лишней информацией. Отсюда чем больше разнообразие обучающих примеров (много коротких окон), тем лучше модель учится различать стили разных авторов.

В ходе экспериментов с различными размерами словаря токенизатора (VOCAB_SIZE) было выявлено, что наибольшая точность классификации — 71.5% — достигается при максимальном размере словаря в 40,000 слов. Это свидетельствует о том, что увеличение размера словаря позволяет модели лучше охватывать разнообразие лексики и тем самым эффективнее распознавать авторов текстов. При этом минимальный размер словаря в 5,000 слов обеспечил точность 67.75%, что говорит о том, что даже небольшой словарь может сохранять важную информацию для классификации, однако охват слов становится ограничен. Интересный факт заключается в том, что при размере словаря 10,000 точность заметно снизилась до 60.81%, что может быть связано с особенностями распределения слов и их частоты в корпусе. При размере 20,000 слов точность немного улучшилась до 63.26%, но все еще уступала результату при 40,000.